# Library

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.svm import SVC, SVR
from sklearn.ensemble import (
    RandomForestRegressor,
    RandomForestClassifier,
    VotingRegressor,
    StackingRegressor,
    GradientBoostingRegressor,
    BaggingRegressor,
    VotingClassifier,
    StackingClassifier,
    BaggingClassifier,
    GradientBoostingClassifier
)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_graphviz

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import graphviz

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.linear_model import LogisticRegression

# Ensemble

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61f7bbd4e90cce440b88ea32_ensemble-learning.png" width="600" height="300"/>

<br>

두 개 이상의 모델을 결합하여 task를 수행하는 기법 <br> 
오차의 분산을 줄이기에 개별 모델보다 강건함(robust) <br>
    -> 한 모델에서 포착하지 못했던 정보를 다른 모델에서 포착할 수 있음 <br>
따라서 모델 간 다양성이 클수록 ensemble 모델의 효과가 커짐 <br>

<br>

<font style="font-size:20px"> 아래와 같은 상황에서 사용 </font> <p>
1. 모델 선택의 어려움: 데이터셋 내의 다양한 부분에서 서로 다른 모델들이 강점을 보이는 경우, 여러 모델을 앙상블하여 각 클래스에 대한 강점을 결합함으로써 전체적인 성능 향상
2. 문제의 복잡성: 단일 분류기로 처리하기 어려운 복잡한 결정 경계를 다룰 때, 여러 분류기의 앙상블은 보다 복잡한 결정 경계 근사
3. 데이터 양: 충분한 데이터가 있는 경우, 단일 분류기보다 여러 분류기를 사용하여 작업을 분할하고 예측 시에 앙상블하는 것이 더 효과적일 수 있음. 반대로, 데이터가 제한적인 경우, 부트스트래핑(데이터의 여러 부분집합에서 다양한 분류기를 훈련) 전략 사용

<br>

<font style="font-size:20px"> 앙상블을 통한 이점 </font> <p>
1. 예측 신뢰도: 앙상블은 각 개별 모델의 신뢰도에 의존하기에, 높은 신뢰도를 가진 모델의 예측을 결합함으로써 단순한 과반수 투표보다 전체적인 예측 신뢰성을 향상
2. 정보 통합: 앙상블은 동일한 클래스에 관련된 다양한 데이터셋이나 조건에서 훈련된 모델들의 예측을 통합하여 분류의 견고성을 향상시키고 편향을 줄임

## Voting

<img src="https://www.researchgate.net/publication/362311832/figure/fig3/AS:11431281078760466@1660240577836/The-figure-shows-with-an-example-how-both-hard-and-soft-voting-work-The-ensemble.ppm" width="600" height="300"/>

모델의 결과를 취합하여 성능 향상을 취하는 방법 <br>
hard voting과 soft voting 두 방법 존재 <br>

<br>

<font style="font-size:20px"> Hard Voting </font> <p>
각 모델에서의 결과를 다수결로 결정하는 방법

<br>

<font style="font-size:20px"> Soft Voting </font> <p>
각 모델에서 결과를 확률로 받은 후 확률의 평균 등을 이용하여 최종 결과 결정

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> from sklearn.ensemble import VotingRegressor
> 
> voting_regressor = VotingRegressor([
>   ('model1', model1),
>   ('model2', model2),
>   ('model3', model3),
> ])
> 
> voting_regressor.fit(X, y)    # train
> voting_regressor.predict(X)   # predict
> ```

<br>

주요 파라미터
- estimators: base learners
- weights: base learner 결과에 계산할 가중치

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [2]:
def convert_category_into_integer(df: pd.DataFrame, columns: list):
    label_encoders = {}
    for column in columns:
        label_encoder = LabelEncoder()
        df.loc[:, column] = label_encoder.fit_transform(df[column])

        label_encoders.update({column: label_encoder})
    
    return df, label_encoders

In [12]:
diamonds, _ = convert_category_into_integer(diamonds, ('cut', 'color', 'clarity'))
diamonds = diamonds.drop_duplicates().reset_index(drop=True)

train, test = train_test_split(diamonds, test_size=0.2, random_state=0)
diamonds


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53789,0.72,2,0,2,60.8,57.0,2757,5.75,5.76,3.50
53790,0.72,1,0,2,63.1,55.0,2757,5.69,5.75,3.61
53791,0.70,4,0,2,62.8,60.0,2757,5.66,5.68,3.56
53792,0.86,3,4,3,61.0,58.0,2757,6.15,6.12,3.74


In [21]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(train.drop(columns=['cut']), train.cut)

DecisionTreeClassifier(random_state=0)

In [22]:
(tree.predict(test.drop(columns=['cut'])) == test['cut']).mean()

0.708523096942095

In [33]:
voting_classifier = VotingClassifier(
    [(f'model{i+1}', DecisionTreeClassifier(random_state=i)) for i in range(10)]
)

voting_classifier.fit(train.drop(columns=['cut']), train['cut'])

(voting_classifier.predict(test.drop(columns=['cut'])) == test['cut']).mean()

0.713263314434427

In [5]:
# diamonds dataset
# 1. 결측치, 중복값 확인
diamonds = sns.load_dataset('diamonds')
diamonds = diamonds.dropna().drop_duplicates().reset_index(drop=True)

# 2. train, test split
diamonds, _ = convert_category_into_integer(diamonds, ('cut', 'color', 'clarity'))
diamonds = diamonds.drop_duplicates().reset_index(drop=True)

train, test = train_test_split(diamonds, test_size=0.2, random_state=0)
diamonds

# 3. train 데이터로 scaling fit 후 train test scaling
standard_scaler = StandardScaler()
target_columns = train.columns.difference(['cut', 'color', 'clarity'])

standard_scaler.fit(train.loc[:, target_columns])
train.loc[:, target_columns] = standard_scaler.transform(train.loc[:, target_columns])
test.loc[:, target_columns] = standard_scaler.transform(test.loc[:, target_columns])

#display(train)

# 4. SVM, Logistic 객체 정의
svm = SVC(random_state=0)
svm.fit(train.drop(columns=['cut']), train.cut)

logistic = LogisticRegression(random_state=0)
logistic.fit(train.drop(columns=['cut']), train.cut)

# 5. SVM, Logistic 개별적으로 학습후 각 모델의 성능 확인

print(f'svm acc: {(svm.predict(test.drop(columns=['cut'])) == test['cut']).mean()}')
print(f'logistic acc: {(logistic.predict(test.drop(columns=['cut'])) == test['cut']).mean()}')

# 6. voting 분류기 정의
voting_classifier = VotingClassifier([
    ('model1', SVC(random_state=0)),
    ('model2', LogisticRegression(random_state=0))
])

# 7. voting 분류기 학습 후 성능 확인
voting_classifier.fit(train.drop(columns=['cut']), train.cut)

# 8. 개별 모델 성능과 voting 모델 성능비교
print(f'voting_classifier acc: {(voting_classifier.predict(test.drop(columns=['cut'])) == test['cut']).mean()}')

C:\Users\USER\AppData\Local\Temp\ipykernel_4328\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_4328\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_4328\1150649826.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.


svm acc: 0.72516033088577
logistic acc: 0.6472720513058835


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


voting_classifier acc: 0.6917929175573938


In [17]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53794 entries, 0 to 53793
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53794 non-null  float64
 1   cut      53794 non-null  int32  
 2   color    53794 non-null  int32  
 3   clarity  53794 non-null  int32  
 4   depth    53794 non-null  float64
 5   table    53794 non-null  float64
 6   price    53794 non-null  int64  
 7   x        53794 non-null  float64
 8   y        53794 non-null  float64
 9   z        53794 non-null  float64
dtypes: float64(6), int32(3), int64(1)
memory usage: 3.5 MB


In [9]:
voting_classifier = VotingClassifier([
    ('model1', SVC(random_state=0, kernel='linear', probability= True)),
    ('model2', LogisticRegression(random_state=0, max_iter= 1000))
],
voting='soft')

# 7. voting 분류기 학습 후 성능 확인
voting_classifier.fit(train.drop(columns=['cut']), train.cut)

# 8. 개별 모델 성능과 voting 모델 성능비교
print(f'voting_classifier acc: {(voting_classifier.predict(test.drop(columns=['cut'])) == test['cut']).mean()}')

voting_classifier acc: 0.6619574309880101


## Stacking

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414dba9e9f94d7b31368_RhQ6ctlYepNo3J-yChyk_jLM_siHT9eGIJTpcI0NEPhADEcGic31JW4TWwLLzWv0LvqyDjFx9yQ8m16kKENTtPZeW-fY-9z6k7m-rsmPseGIeHhB-IiI0V5t4hImEPZRnEWPChAo.png" width="600" height="300"/>

<br>
<br>

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*T-JHq4AK3dyRNi7gpn9-Xw.png" width="600" height="300"/>

<br>

base 모델 학습과 meta model 학습의 두 가지 단계를 통하여 ensemble 모델 구축 <br>
base learner는 동일 데이터에서 여러 모델을 학습하거나 혹은 bagging 방법을 통하여 구축 <br>
이렇게 학습한 모델을의 결과를 meta learner라고 하는 모델의 input으로 사용하여 meta learner 학습 <br>

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> from sklearn.ensemble import StackingRegressor
> 
> base_learners = [
>   ('model1', model1),
>   ('model2', model2),
>   ('model3', model3),
> ]
> meta_learner = model4
> 
> stacking_regressor = StackingRegressor(
>   estimators=base_learners,
>   final_estimator=meta_learner,
> )
> 
> stacking_regressor.fit(X, y)    # train
> stacking_regressor.predict(X)   # predict
> ```

<br>

주요 파라미터
- estimators: base learners
- final_estimator: meta learner
- cv: num of folds (cross-validation)

In [23]:
penguins = sns.load_dataset('penguins')
penguins

# 1. 결측치, 중복값 확인

penguins = penguins.dropna().drop_duplicates().reset_index(drop=True)

penguins, _ = convert_category_into_integer(penguins, ('species', 'island', 'sex'))

penguins.species = penguins.species.astype(int)
penguins.island = penguins.island.astype(int)
penguins.sex = penguins.sex.astype(int)

train, test = train_test_split(penguins, test_size=0.3, random_state=0)
#penguins.info()
standard_scaler = StandardScaler()

target_columns = train.columns.difference(['species', 'island', 'sex'])
standard_scaler.fit(train.loc[:, target_columns])
train.loc[:, target_columns] = standard_scaler.transform(train.loc[:, target_columns])
test.loc[:, target_columns] = standard_scaler.transform(test.loc[:, target_columns])


svc = SVC(kernel='linear', random_state=0)
logistic = LogisticRegression(max_iter=1000, random_state=0)
decision_tree = DecisionTreeClassifier(random_state=0)

svc.fit(train.drop(columns=['species']), train.species)
logistic.fit(train.drop(columns=['species']), train.species)
decision_tree.fit(train.drop(columns=['species']), train.species)

print(f'svc acc: {(svc.predict(test.drop(columns=['species'])) == test['species']).mean()}')
print(f'logistic acc: {(logistic.predict(test.drop(columns=['species'])) == test['species']).mean()}')
print(f'decision_tree acc: {(decision_tree.predict(test.drop(columns=['species'])) == test['species']).mean()}')

base_learners = [
    ('model1', SVC(kernel='linear', random_state=0)),
    ('model2', LogisticRegression(max_iter=1000, random_state=0)),
    ('model3', DecisionTreeClassifier(random_state=0)),
]

meta_learner = RandomForestClassifier(random_state=0)

stacking_classifier = StackingClassifier(
  estimators = base_learners,
  final_estimator=meta_learner,
)

stacking_classifier.fit(train.drop(columns=['species']), train.species)


print(f'stacking_classifier acc: {(stacking_classifier.predict(test.drop(columns=['species'])) == test['species']).mean()}')

svc acc: 0.98
logistic acc: 0.98
decision_tree acc: 0.98
stacking_classifier acc: 0.98


In [ ]:
base_learners = [
>   ('model1', model1),
>   ('model2', model2),
>   ('model3', model3),
> ]
> meta_learner = model4
> 
> stacking_regressor = StackingRegressor(
>   estimators=base_learners,
>   final_estimator=meta_learner,
> )
> 
> stacking_regressor.fit(X, y)    # train
> stacking_regressor.predict(X

## Boosting

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414d5e568a661fb7896c_mji7xyiAlyQAdxQde14HY1OVvAVzDyyKhDOo4a4bg53_m2OHUvHhMGexaHuHCfKGRVQQlfFlihuodX7LD5hugPgGw8ZzJV4bHjHc648Zr0LyVr2I0i6ciJvJri_OFCuQpOf81xcn.png" width="600" height="300"/>

<br>

데이터 병렬 처리 대신 순차 처리 발생. <br>
주어진 데이터를 첫 번째 모델로 학습. <br>
첫 번째 모델에 의한 오차를 두 번째 모델에서 학습. <br>
두 번째 모델은 첫 번째 모델이 못 맞춘 부분만을 학습. <br>
위의 작업을 반복하여 개별 모델을 학습 후 ensemble하여 최종 모델 구축. <br>

<br>

<font style="font-size:20px"> 대표 알고리즘 </font> <p>
- Adaptive Boosting (AdaBoost)
- XGBoost
- LightGBM
- CatBoost

## Mixture of Experts (MoE)

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414da1649f856e553d74_jiy3kw7cLPoe3-DClXGWwEZo4FNQuf3NzqSpelWXN0di_Ydnyz9QnHqGiFAd87pc-WELMSXGKKdw1wqeA5pjSioytSVNXCmU6wdaV3nFUYZjkKKs_deV_XyUjLOfU8K9o5RWu_n4.png" width="600" height="300"/>

여러 전문화된 하위 모델(전문가)를 조합하여 복잡하고 다양한 패턴을 학습하도록 하는 방법. <br>
기존 단일 모델이 모든 것을 잘 하도록 학습하는 방식에서 모델 별로 잘 하는 분야를 두고 전문화된 모델을 학습. <br>
이러한 모델을 조합하여 최종적인 모델을 구축. <br>
입력 데이터에 따라 각 모델(전문가)의 중요도를 산출하는 gating network를 통해 비중 설정 <br>

<br>

최근 자연어 처리에서 각광받는 GPT도 해당 로직을 탑재한 것으로 추정됨

## Bagging

<img src="https://cdn.prod.website-files.com/5d7b77b063a9066d83e1209c/61a4414d28946a3ac3e69ed9_q-FrlRMLk-5nSxZ_3ONlFpu5hQ61PsuAxkusTD1vEX5NqkdH2Ie0u_75rIySTZKXVI4VBxM-AIw3APQvRboG3kv-3l3cA5c5qyMwwTMe2OLXzoAgA051Dqbx7XVfdJaDyNwrSLUf.png" width="600" height="300"/>

<br>

Boostrap AGGregatING의 약자로 부트스트랩을 통한 방법 <br>
전체 집합에서 일부 데이터를 선택하여 subset을 만든 후(복원추출) 개별 모델을 학습 <br>
-> subset을 여러 개 만들어 동시에 학습할 수 있기에 병렬화 가능
최종 예측은 평균 등의 방법으로 산출 <br>
이 때 사용되는 모델은 **모두 동일한 모델**이어야 함

<br>

<font style="font-size:20px"> 대표 알고리즘 </font> <p>
- Random Forest

<br>

<font style="font-size:20px"> 사용 방법 </font> <p>

> ```python
> from sklearn.ensemble import BaggingRegressor
> 
> bagging_regressor = BaggingRegressor(
>   estimator=model,
> )
> 
> bagging_regressor.fit(X, y)    # train
> bagging_regressor.predict(X)   # predict
> ```

<br>

주요 파라미터
- estimator: base model
- n_estimators: base learner의 수
- random_state: 난수 고정을 위한 seed

In [ ]:
bagging_classifier = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=10),
).fit(train.drop(columns=['cut']), train['cut'])

In [ ]:
print(f'train_acc: {(bagging_classifier.predict(train.drop(columns=['cut'])) == train.cut).mean()}')
print(f'test_acc: {(bagging_classifier.predict(test.drop(columns=['cut'])) == test.cut).mean()}')